In [2]:
import pandas as pd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

import matplotlib.pyplot as plt
import plotly_express as px

import tqdm
from tqdm.notebook import tqdm_notebook
from tqdm.auto import tqdm

In [3]:
#Create the locator information and test if we get an accurate result
locator = Nominatim(user_agent="myGeocoder")
coordinates = "40.8, -124"
location = locator.reverse(coordinates, addressdetails=False, zoom=8)
location.raw
county = location[0].split(',')
county = county[0]
print(county)

Humboldt County


In [4]:
#Read in the fire data for reverse geo. This was also done to the lightning_strikes file. Steps are identical
df = pd.read_csv("fires.csv")
df.head()

,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,...,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME,Shape
0,17,17,FS-1418878,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,G,38.523333,-120.211667,5,USFS,CA,5,5.0,Amador,NaN
1,18,18,FS-1418881,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,G,38.780000,-120.260000,5,USFS,CA,17,17.0,El Dorado,NaN
2,549,556,FS-1420078,FED,FS-FIRESTAT,FS,USCASQF,Sequoia National Forest,0513,Sequoia National Forest,...,F,35.688056,-118.438333,5,USFS,CA,NaN,NaN,NaN,NaN
3,1500,1514,FS-1422180,FED,FS-FIRESTAT,FS,USCACNF,Cleveland National Forest,0502,Cleveland National Forest,...,D,32.681944,-116.640000,5,USFS,CA,NaN,NaN,NaN,NaN
4,2723,2740,FS-1424833,FED,FS-FIRESTAT,FS,USCAMNF,Mendocino National Forest,0508,Mendocino National Forest,...,E,39.169722,-122.887222,14,MISSING/NOT SPECIFIED,CA,NaN,NaN,NaN,NaN


In [5]:
#Combine the latitude and longitude into a format that geopy can read and test it
df["geom"] = df["LATITUDE"].map(str) + ',' + df['LONGITUDE'].map(str)
df["geom"][0]

'38.52333333,-120.21166667'

In [6]:
#Set timeout parameters and create a progress bar. Lightning took 30 hours to finish; fires took about 30 minutes
locator = Nominatim(user_agent="myGeocoder", timeout = 20)
rgeocode = RateLimiter(locator.reverse, min_delay_seconds = 0.001)
tqdm.pandas()
df['location'] = df['geom'].progress_apply(rgeocode)
df.head()

C:\Users\Ed\anaconda3\lib\site-packages\tqdm\std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,...,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME,Shape,geom,location
0,17,17,FS-1418878,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,-120.211667,5,USFS,CA,5,5.0,Amador,NaN,"38.52333333,-120.21166667","(Cole Creek Road, Amador, California, United S..."
1,18,18,FS-1418881,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,-120.260000,5,USFS,CA,17,17.0,El Dorado,NaN,"38.78,-120.26","(Pony Express Trail, Kyburz, California, 95720..."
2,549,556,FS-1420078,FED,FS-FIRESTAT,FS,USCASQF,Sequoia National Forest,0513,Sequoia National Forest,...,-118.438333,5,USFS,CA,NaN,NaN,NaN,NaN,"35.68805556,-118.43833333","(Wofford Heights, Kern County, California, 932..."
3,1500,1514,FS-1422180,FED,FS-FIRESTAT,FS,USCACNF,Cleveland National Forest,0502,Cleveland National Forest,...,-116.640000,5,USFS,CA,NaN,NaN,NaN,NaN,"32.68194444,-116.64","(San Diego County, California, United States o..."
4,2723,2740,FS-1424833,FED,FS-FIRESTAT,FS,USCAMNF,Mendocino National Forest,0508,Mendocino National Forest,...,-122.887222,14,MISSING/NOT SPECIFIED,CA,NaN,NaN,NaN,NaN,"39.16972222,-122.88722222","(Clover Valley Road, Nice, Upper Lake, Lake Co..."


In [7]:
#Write the new fires file to csv
df.to_csv(r'FiresWithCounty.csv', index = False)

In [8]:
df.head()

,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,...,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME,Shape,geom,location
0,17,17,FS-1418878,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,-120.211667,5,USFS,CA,5,5.0,Amador,NaN,"38.52333333,-120.21166667","(Cole Creek Road, Amador, California, United S..."
1,18,18,FS-1418881,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,-120.260000,5,USFS,CA,17,17.0,El Dorado,NaN,"38.78,-120.26","(Pony Express Trail, Kyburz, California, 95720..."
2,549,556,FS-1420078,FED,FS-FIRESTAT,FS,USCASQF,Sequoia National Forest,0513,Sequoia National Forest,...,-118.438333,5,USFS,CA,NaN,NaN,NaN,NaN,"35.68805556,-118.43833333","(Wofford Heights, Kern County, California, 932..."
3,1500,1514,FS-1422180,FED,FS-FIRESTAT,FS,USCACNF,Cleveland National Forest,0502,Cleveland National Forest,...,-116.640000,5,USFS,CA,NaN,NaN,NaN,NaN,"32.68194444,-116.64","(San Diego County, California, United States o..."
4,2723,2740,FS-1424833,FED,FS-FIRESTAT,FS,USCAMNF,Mendocino National Forest,0508,Mendocino National Forest,...,-122.887222,14,MISSING/NOT SPECIFIED,CA,NaN,NaN,NaN,NaN,"39.16972222,-122.88722222","(Clover Valley Road, Nice, Upper Lake, Lake Co..."
